# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> implement any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`bucket()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1616430155178_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [14]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

from pyspark.sql.functions import col
from pyspark.sql import *

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-spring2021"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp,lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [4]:
def user():
    # Returns a string consisting of your GT username.
    return 'fjpm3'
   

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [5]:
def long_trips(trips):
    # Returns a Dataframe with Schema the same as :trips:
   # trips.where(trips >= 2)
    LT = trips.where(col("trip_distance") >= 2.0)
    return LT
    pass  #trips.where(col("trips") >= 2).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [6]:
def manhattan_trips(trips, lookup):
    # Returns a Dataframe with Schema: DOLocationID, pcount 
    matter = trips
    manh = lookup.join(matter,matter["DOLocationID"] == lookup["LocationID"]).withColumn("passenger_count",col("passenger_count").cast("int"))\
    .groupBy("DOLocationID","passenger_count").sum("passenger_count").withColumnRenamed("sum(passenger_count)","pcount")\
    .select(col("DOLocationID"),col("pcount")).orderBy(col("pcount").desc()).limit(20)
     
    return manh
    pass 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [7]:
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    matter2 = trips#.withColumnRenamed("DOLocationID","MALocation")
    
    wp1 = matter2\
    .withColumn("passenger_count",col("passenger_count").cast("int"))\
    .groupBy("PULocationID","DOLocationID","passenger_count").agg({"passenger_count":"avg","PULocationID":"count"})\
    .withColumnRenamed("avg(passenger_count)","avgpu").withColumnRenamed("count(PULocationID)","pucount")\
    .select("PULocationID","DOLocationID","avgpu","pucount").orderBy(col("PULocationID").desc())
    matrips = mtrips.withColumnRenamed("DOLocationID","MALocation")
    
    wp2 = wp1.join(matrips,matrips["MALocation"] == wp1["DOLocationID"])\
    .select("PULocationID","MALocation")\
    .groupBy("PULocationID").agg({"MALocation":"count"}).withColumnRenamed("count(MALocation)","macount")\
    .groupBy("PULocationID").agg({"macount":"sum"}).withColumnRenamed("PULocationID","LocationID").select("LocationID","sum(macount)")
    wp3 = wp1.groupBy("PULocationID").agg({"avgpu":"avg","pucount":"sum"}).select("PULocationID","sum(pucount)","avg(avgpu)")
    wp4 = wp3.join(wp2,wp2["LocationID"] == wp3["PULocationID"]).drop("LocationID").select("PULocationID",((col("sum(macount)")/col("sum(pucount)"))*col("avg(avgpu)")))\
    .withColumnRenamed("((sum(macount) / sum(pucount)) * avg(avgpu))","weighted_profit")\
    .select("PULocationID","weighted_profit")
    return wp4
    pass 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [12]:
def final_output(calc, lookup): 
    # Returns a Dataframe with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    calc1 = calc.withColumnRenamed("PULocationID","Location")
    fin = calc1.join(lookup,calc1["Location"] == lookup["LocationID"]).select("Zone","Borough","weighted_profit").orderBy(col("weighted_profit").desc()).limit(20)
    
    return fin
    pass

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<div class="alert alert-block alert-info">
    Test your code on the small dataset first, as the large dataset will take a significantly longer time to run
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> use the same bucket url for multiple runs of the `main()` function, as this will cause errors. Make sure to change the name of your output location every time. (ie: s3://cse6242-gburdell3/output-small2)
</div>

Update the below cell with the path to your bucket, then run the below cell to run your code to store the results in S3.

When you have confirmed the results of the small dataset, run it again using the large dataset. Your output file will appear in a folder in your s3 bucket called YOUROUTPUT.csv as a csv file with a name something like part-0000-4d992f7a-0ad3-48f8-8c72-0022984e4b50-c000.csv. Download this file and rename it to q3_output.csv for submission. Do not make any other changes to the file. 

In [16]:
# update your bucket path
#bucket = 's3://cse6242-fjpm3/output-small1'
#main('small', bucket)

bucket = 's3://cse6242-fjpm3/output-large'
main('large', bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

fjpm3
Trip Count:  187203269
+--------------------+-------------+-------------------+
|                Zone|      Borough|    weighted_profit|
+--------------------+-------------+-------------------+
|Governor's Island...|    Manhattan| 0.4394690040650407|
|       Rikers Island|        Bronx|0.39999999999999997|
|         Jamaica Bay|       Queens| 0.3916426261036804|
|    Great Kills Park|Staten Island| 0.3333333333333333|
|Eltingville/Annad...|Staten Island|0.32432432432432434|
|     Freshkills Park|Staten Island|               0.26|
|Breezy Point/Fort...|       Queens|0.24807237586100545|
|        Country Club|        Bronx|0.22611343738104303|
|          Bronx Park|        Bronx|0.20241132167152576|
|         Westerleigh|Staten Island|0.17599301615015275|
|              Corona|       Queens|0.17033618984838497|
|       Willets Point|       Queens|0.16853906065612465|
|       Port Richmond|Staten Island|0.16216216216216217|
|    Inwood Hill Park|    Manhattan|0.15392973560304238|
| 

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading
</div>

In [148]:
manhattan_trips(trips, lookup)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'trips' is not defined
Traceback (most recent call last):
NameError: name 'trips' is not defined

